## Import necessary headers

In [147]:
from pynq import Overlay
from pynq import MMIO
import random
import string
import numpy as np
import sys
import hashlib
print("User Current Version:-", sys.version)

User Current Version:- 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]


## Generate input

In [131]:
N = random.randint(1, 1000)
rand_bytes = [random.randint(0, sys.maxsize) for i in range(N)]
zeros = 14 - N % 16
zeros = zeros if zeros >= 0 else zeros + 16
rand_bytes += [0 for i in range(zeros)]
rand_bytes += rand_bytes[0:2]

frags = (len(rand_bytes)) // 16
rand_bytes = np.array(rand_bytes)

print(N, zeros, frags)

618 4 39


## Software SHA-256

In [163]:
k = np.array([
   0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
   0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
   0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
   0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
   0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
   0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
   0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
   0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2], dtype=np.uint32)

def rightRotate(n, d):
    return (n >> d) | (n << (32 - d)) & 0xFFFFFFFF

def sha_256(sha_input, frags):
    acc = np.array([0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
         0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19], dtype=np.uint32)
    for iteration in range(frags):
        w = []
        for i in range(16):
            w.append(sha_input[i * 16 + iteration])
        for i in range(16, 64):
            s0 = rightRotate(w[i - 15], 7) ^ rightRotate(w[i - 15], 18) ^ (w[i-15] >> 3)
            s1 = rightRotate(w[i - 2], 17) ^ rightRotate(w[i - 2], 19) ^ (w[i-2] >> 10)
            w.append(w[i - 16] + s0 + w[i - 7] + s1)
        tmp = acc
        for i in range(64):
            s0 = rightRotate(tmp[0], 2) ^ rightRotate(tmp[0], 13) ^ rightRotate(tmp[0], 22)
            maj = (tmp[0] & tmp[1]) ^ (tmp[0] & tmp[2]) ^ (tmp[1] & tmp[2])
            t2 = s0 + maj
            s1 = rightRotate(tmp[4], 6) ^ rightRotate(tmp[4], 11) ^ rightRotate(tmp[4], 25)
            ch = (tmp[4] & tmp[5]) ^ (~tmp[4] & tmp[6])
            t1 = tmp[7] + s1 + ch + k[i] + w[i]
            
            tmp[7], tmp[6], tmp[5], tmp[4], tmp[3], tmp[2], tmp[1], tmp[0] = \
            tmp[6], tmp[5], tmp[4], (tmp[3] + t1) & 0xffffffff, tmp[2], tmp[1], tmp[0], (t1 + t2) & 0xffffffff
        acc += tmp
    return acc

hash_val = sha_256(rand_bytes, frags)
output = ''
for i in range(8):
    output += str(hex(hash_val[i]))[2:].z
print(output)
print(len(output))

c23872787d8f9e5cc15c74ae15e2911aaffe3570bb724818892e08c615d8b56
63
